In [75]:
import copy
import json
import zstd
import numpy as np

In [50]:
from training_module import hlt

In [4]:
path = '/Users/Peace/Desktop/replays/ts2018-halite-3-gold-replays_replay-20181016-000040%2B0000-1539647993-48-48'

with open(path, 'rb') as infile:
    raw_replay = zstd.loads(infile.read()).decode()
    
parsed_replay = json.loads(raw_replay)

In [6]:
parsed_replay.keys()

dict_keys(['ENGINE_VERSION', 'GAME_CONSTANTS', 'REPLAY_FILE_VERSION', 'full_frames', 'game_statistics', 'map_generator_seed', 'number_of_players', 'players', 'production_map'])

In [16]:
meta_data = {key: parsed_replay[key] for key in ['ENGINE_VERSION', 'GAME_CONSTANTS', 'REPLAY_FILE_VERSION',
                                                'game_statistics', 'map_generator_seed',
                                                'number_of_players', 'players']}

In [37]:
parsed_replay['full_frames'][0].keys()

dict_keys(['cells', 'deposited', 'energy', 'entities', 'events', 'moves'])

In [86]:
parsed_replay['full_frames'][15]#['energy']

{'cells': [{'production': 24, 'x': 13, 'y': 33},
  {'production': 276, 'x': 16, 'y': 17},
  {'production': 302, 'x': 32, 'y': 31},
  {'production': 298, 'x': 16, 'y': 16},
  {'production': 130, 'x': 32, 'y': 15},
  {'production': 15, 'x': 15, 'y': 33},
  {'production': 221, 'x': 13, 'y': 11},
  {'production': 47, 'x': 16, 'y': 33},
  {'production': 338, 'x': 17, 'y': 16},
  {'production': 105, 'x': 31, 'y': 13},
  {'production': 174, 'x': 15, 'y': 32},
  {'production': 234, 'x': 15, 'y': 17}],
 'deposited': {'0': 0, '1': 0, '2': 5, '3': 292},
 'energy': {'0': 0, '1': 0, '2': 5, '3': 292},
 'entities': {'0': {'10': {'energy': 203,
    'is_inspired': False,
    'x': 16,
    'y': 16},
   '12': {'energy': 323, 'is_inspired': False, 'x': 16, 'y': 17},
   '2': {'energy': 286, 'is_inspired': False, 'x': 17, 'y': 16},
   '5': {'energy': 545, 'is_inspired': False, 'x': 15, 'y': 17},
   '7': {'energy': 200, 'is_inspired': False, 'x': 13, 'y': 11}},
  '1': {'15': {'energy': 39, 'is_inspired': Fal

In [22]:
parsed_replay['production_map']['grid'][0][0].keys()

dict_keys(['energy'])

In [33]:
assert list(parsed_replay['production_map']['grid'][0][0].keys()) == ['energy'] # Ensure no surprises here

In [48]:
raw_energy_grid = parsed_replay['production_map']['grid']
energy_grid = []
for row in raw_energy_grid:
    energy_grid.append([x['energy'] for x in row])
np.array(energy_grid)


array([[  6,  14,  48, ...,  48,  14,   6],
       [ 33,  61,  38, ...,  38,  61,  33],
       [160, 118,  23, ...,  23, 118, 160],
       ...,
       [160, 118,  23, ...,  23, 118, 160],
       [ 33,  61,  38, ...,  38,  61,  33],
       [  6,  14,  48, ...,  48,  14,   6]])

In [82]:
def extract_frames(data):
    width = data['production_map']['width']
    height = data['production_map']['height']


    print("Load Cell Information")
    first_cells = []
    for x in range(len(data['production_map']['grid'])):
        row = []
        for y in range(len(data['production_map']['grid'][x])):
            row += [hlt.MapCell(hlt.Position(x, y), data['production_map']['grid'][x][y]['energy'])]
        first_cells.append(row)
    frames = []
    for f in data['full_frames']:
        prev_cells = first_cells if len(frames) == 0 else frames[-1]._cells
        new_cells = copy.deepcopy(prev_cells)
        for c in f['cells']:
            new_cells[c['y']][c['x']].halite_amount = c['production']
        frames.append(hlt.GameMap(new_cells, width, height))
    
    return frames

In [76]:
def parse_replay(data, player_names):
    
    for player in player_names:

        print("Load Basic Information")
        player = [p for p in data['players'] if p['name'].split(" ")[0] == player_name][0]
        player_id = int(player['player_id'])
        my_shipyard = hlt.Shipyard(player_id, None,
                                   hlt.Position(player['factory_location']['x'], player['factory_location']['y']))
        other_shipyards = [
            hlt.Shipyard(p['player_id'], None, hlt.Position(p['factory_location']['x'], p['factory_location']['y']))
            for p in data['players'] if int(p['player_id']) != player_id]
        width = data['production_map']['width']
        height = data['production_map']['height']



        print("Load Player Ships")
        moves = [{} if str(player_id) not in f['moves'] else {m['id']: m['direction'] for m in f['moves'][str(player_id)] if
                                                              m['type'] == "m"} for f in data['full_frames']]
        ships = [{} if str(player_id) not in f['entities'] else {
            int(sid): hlt.Ship(player_id, int(sid), hlt.Position(ship['x'], ship['y']), ship['energy']) for sid, ship in
            f['entities'][str(player_id)].items()} for f in data['full_frames']]

        print("Load Other Player Ships")
        other_ships = [
            {int(sid): hlt.Ship(int(pid), int(sid), hlt.Position(ship['x'], ship['y']), ship['energy']) for pid, p in
             f['entities'].items() if
             int(pid) != player_id for sid, ship in p.items()} for f in data['full_frames']]

        print("Load Droppoff Information")
        first_my_dropoffs = [my_shipyard]
        first_them_dropoffs = other_shipyards
        my_dropoffs = []
        them_dropoffs = []
        for f in data['full_frames']:
            new_my_dropoffs = copy.deepcopy(first_my_dropoffs if len(my_dropoffs) == 0 else my_dropoffs[-1])
            new_them_dropoffs = copy.deepcopy(first_them_dropoffs if len(them_dropoffs) == 0 else them_dropoffs[-1])
            for e in f['events']:
                if e['type'] == 'construct':
                    if int(e['owner_id']) == player_id:
                        new_my_dropoffs.append(
                            hlt.Dropoff(player_id, None, hlt.Position(e['location']['x'], e['location']['y'])))
                    else:
                        new_them_dropoffs.append(
                            hlt.Dropoff(e['owner_id'], None, hlt.Position(e['location']['x'], e['location']['y'])))
            my_dropoffs.append(new_my_dropoffs)
            them_dropoffs.append(new_them_dropoffs)
        yield list(zip(frames, moves, ships, other_ships, my_dropoffs, them_dropoffs))

In [77]:
player_names = [x['name'].split(' ')[0] for x in meta_data['players']]

In [83]:
frames = extract_frames(parsed_replay)

Load Cell Information


In [84]:
frames[:2]

In [79]:
#all_data = []
_data = parse_replay(parsed_replay, player_names)


Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information
